<a href="https://colab.research.google.com/github/mimrancomsats/ProgrammingforAI_FALL25/blob/main/Lab_9_MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MLFlow**

**MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in handling the complexities of the machine learning process.**

**In this notebook, we are going to use MLFlow for Experiment Tracking**

**The installation process of MLFlow is described in the following link:**




https://mlflow.org/docs/latest/getting-started/intro-quickstart/index.html

# **MLFlow Library Installation**

In [1]:
!pip install --quiet mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.2 MB/s eta 0:00:00


# **Sklearn Pipeline Implementation (KNN)**

In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
titanic_data = pd.read_csv('titanic.csv')

# Custom function to impute missing values in 'Embarked' column
def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    return X

# Custom function to create the 'FamilySize' feature
def create_family_size(X):
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1  # Add 1 for the individual themselves
    return X

# Custom function to drop columns that are not needed for model training
def drop_columns(X):
    return X.drop(['SibSp', 'Parch'], axis=1)

# Function to create 'FamilySize' and drop 'SibSp' and 'Parch' columns
def family_size(X):
    #print(X)
    X = create_family_size(X)
    #print(X)
    X = drop_columns(X)
    #print(X)
    return X

# Pipeline to preprocess 'Age' column
age_pipeline = Pipeline(steps=[
    ('age_imputer', SimpleImputer(strategy='mean')),  # Impute missing 'Age' values
    ('age_scaler', MinMaxScaler())  # Scale 'Age' feature
])

# Pipeline to preprocess 'Fare' column
fare_pipeline = Pipeline(steps=[
    #('fare_imputer', SimpleImputer(strategy='mean')),  # Optionally impute missing 'Fare'
    ('fare_scaler', MinMaxScaler())  # Scale 'Fare' feature
])

# Pipeline to create and scale the 'FamilySize' feature
family_size_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(family_size)),
    ('family_size_scaler', MinMaxScaler()),  # Scale 'FamilySize'
])

# Pipeline to preprocess 'Embarked' column
embarked_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked)),  # Impute missing 'Embarked' values
    ('embarked_onehot', OneHotEncoder())  # One-hot encode 'Embarked'
])

# Create a ColumnTransformer to preprocess all relevant features
knn_preprocessor = ColumnTransformer(transformers=[
    ('age_encoder', age_pipeline, ['Age']),  # Preprocess 'Age'
    ('fare_encoder', fare_pipeline, ['Fare']),  # Preprocess 'Fare'
    ('family_size', family_size_pipeline, ['SibSp', 'Parch']),  # Preprocess 'FamilySize'
    ('embarked_encoder', embarked_pipeline, ['Embarked']),  # Preprocess 'Embarked'
    ('sex_encoder', OneHotEncoder(), ['Sex']),  # One-hot encode 'Sex'
    ('pclass_scaler', MinMaxScaler(), ['Pclass']),  # Scale 'Pclass'
], remainder='passthrough')

# Create a complete pipeline with preprocessing and the KNN classifier
knn_pipeline = Pipeline(steps=[
    ('knn_preprocessor', knn_preprocessor),  # Data preprocessing steps
    ('knn_classifier', KNeighborsClassifier(n_neighbors=5))  # KNN Classifier
])

# Separate features and target variable
#X = data.drop('Survived', axis=1)
X = titanic_data.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = titanic_data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
knn_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_pipeline.predict(X_test)

# Evaluate the model performance
knn_accuracy = accuracy_score(y_test, y_pred)
print(f"\nKNN Model Accuracy: {knn_accuracy:.2f}")

# Confusion matrix for evaluating the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



KNN Model Accuracy: 0.80
Confusion Matrix:
[[90 15]
 [21 53]]


# **Experiment Tracking**

In [4]:
import mlflow
import mlflow.sklearn

# Set the tracking URI and experiment name
mlflow.set_tracking_uri(uri="http://54.172.101.254:5000")
mlflow.set_experiment("KNN Experiment")

# Start a new MLflow run
with mlflow.start_run():

    # Log the prameters related to KNN model
    mlflow.log_param("model","KNN")
    mlflow.log_param("n_neighbors", 5)
    mlflow.log_param("metric", 'euclidean')

    # Log the accuracy metric
    mlflow.log_metric("accuracy", knn_accuracy)

    # Log the KNN model (use the knn_pipeline variable)
    mlflow.sklearn.log_model(knn_pipeline, "KNN Algorithm")


2025/11/19 13:30:40 INFO mlflow.tracking.fluent: Experiment with name 'KNN Experiment' does not exist. Creating a new experiment.
2025/11/19 13:30:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 13:30:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run intelligent-colt-7 at: http://54.172.101.254:5000/#/experiments/978479126985052510/runs/5883ae5f5caa4eae9eba7c4e85be2e8c
🧪 View experiment at: http://54.172.101.254:5000/#/experiments/978479126985052510


# **Sklearn Pipeline Implementation (Decision Tree)**

In [5]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the dataset
data = pd.read_csv('titanic.csv')

# Custom function to impute missing values in the 'Embarked' column
def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    return X

# Custom function to create the 'FamilySize' feature
def create_family_size(X):
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1  # Adding 1 for the individual themselves
    return X

# Custom function to drop specified columns
def drop_columns(X):
    return X.drop(['SibSp', 'Parch'], axis=1)

# Function to create 'FamilySize' and drop 'SibSp' and 'Parch' columns
def family_size(X):
    X = create_family_size(X)
    X = drop_columns(X)
    return X

# Create pipelines for 'Age'
age_pipeline = Pipeline(steps=[
    ('age_imputer', SimpleImputer(strategy='mean')),  # Impute missing 'Age' values
    ('age_scaler', MinMaxScaler())  # Scale 'Age' feature
])

# Create pipelines for 'Fare'
fare_pipeline = Pipeline(steps=[
    ('fare_scaler', MinMaxScaler())  # Scale 'Fare' feature
])

# Create pipelines for 'FamilySize'
family_size_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(family_size)),
    ('family_size_scaler', MinMaxScaler())  # Scale 'FamilySize' feature
])

# Create pipelines for 'Embarked'
embarked_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked)),  # Impute missing 'Embarked' values
    ('embarked_onehot', OneHotEncoder())  # One-hot encode 'Embarked'
])

# Create a ColumnTransformer to preprocess the data
dt_preprocessor = ColumnTransformer(transformers=[
    ('drop_columns', 'drop', ['PassengerId', 'Name', 'Ticket', 'Cabin']),  # Drop irrelevant columns
    ('age_encoder', age_pipeline, ['Age']),  # Preprocess 'Age'
    ('fare_encoder', fare_pipeline, ['Fare']),  # Preprocess 'Fare'
    ('family_size', family_size_pipeline, ['SibSp', 'Parch']),  # Preprocess 'FamilySize'
    ('embarked_encoder', embarked_pipeline, ['Embarked']),  # Preprocess 'Embarked'
    ('sex_encoder', OneHotEncoder(), ['Sex']),  # One-hot encode 'Sex'
    ('pclass_scaler', MinMaxScaler(), ['Pclass']),  # Scale 'Pclass'
], remainder='passthrough')

# Create a complete pipeline that includes preprocessing and the Decision Tree classifier
dt_pipeline = Pipeline(steps=[
    ('dt_preprocessor', dt_preprocessor),  # Data preprocessing steps
    ('dt_classifier', DecisionTreeClassifier(criterion='entropy'))  # Decision Tree Classifier
])

# Separate features and target variable
X = data.drop('Survived', axis=1)
y = data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
dt_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dt_pipeline.predict(X_test)

# Evaluate the model performance
dt_accuracy = accuracy_score(y_test, y_pred)
print(f"\nDecision Tree Model Accuracy: {dt_accuracy:.2f}")

# Confusion matrix for evaluating the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Decision Tree Model Accuracy: 0.79
Confusion Matrix:
[[87 18]
 [20 54]]


# **Experiment Tracking**

In [6]:
import mlflow
import mlflow.sklearn

# Set the tracking URI and experiment name
mlflow.set_tracking_uri(uri="http://54.172.101.254:5000")
mlflow.set_experiment("Decision Tree Experiment")

# Start a new MLflow run
with mlflow.start_run():

    # Log the parameters related to Decision Tree model
    mlflow.log_param("model","Decision Tree")
    mlflow.log_param("criterion", "entropy")

    # Log the accuracy metric
    mlflow.log_metric("accuracy", dt_accuracy)

    # Log the Decision Tree model (use the dt_pipeline variable)
    mlflow.sklearn.log_model(dt_pipeline, "Decision Tree Algorithm")


2025/11/19 13:35:03 INFO mlflow.tracking.fluent: Experiment with name 'Decision Tree Experiment' does not exist. Creating a new experiment.
2025/11/19 13:35:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 13:35:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rogue-moose-799 at: http://54.172.101.254:5000/#/experiments/329376391994312541/runs/520ef0f06e1e4c77b812db6dcb4d3f0a
🧪 View experiment at: http://54.172.101.254:5000/#/experiments/329376391994312541


# **Sklearn Pipeline Implementation (Random Forest)**

In [7]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the dataset
data = pd.read_csv('titanic.csv')

# Custom function to impute missing values in the 'Embarked' column
def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    return X

# Custom function to create the 'FamilySize' feature
def create_family_size(X):
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1  # Adding 1 for the individual themselves
    return X

# Custom function to drop specified columns
def drop_columns(X):
    return X.drop(['SibSp', 'Parch'], axis=1)

# Function to create 'FamilySize' and drop 'SibSp' and 'Parch' columns
def family_size(X):
    X = create_family_size(X)
    X = drop_columns(X)
    return X

# Create pipelines for 'Age'
age_pipeline = Pipeline(steps=[
    ('age_imputer', SimpleImputer(strategy='mean')),  # Impute missing 'Age' values
    ('age_scaler', MinMaxScaler())  # Scale 'Age' feature
])

# Create pipelines for 'Fare'
fare_pipeline = Pipeline(steps=[
    ('fare_scaler', MinMaxScaler())  # Scale 'Fare' feature
])

# Create pipelines for 'FamilySize'
family_size_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(family_size)),
    ('family_size_scaler', MinMaxScaler())  # Scale 'FamilySize' feature
])

# Create pipelines for 'Embarked'
embarked_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked)),  # Impute missing 'Embarked' values
    ('embarked_onehot', OneHotEncoder())  # One-hot encode 'Embarked'
])

# Create a ColumnTransformer to preprocess the data
rf_preprocessor = ColumnTransformer(transformers=[
    ('drop_columns', 'drop', ['PassengerId', 'Name', 'Ticket', 'Cabin']),  # Drop irrelevant columns
    ('age_encoder', age_pipeline, ['Age']),  # Preprocess 'Age'
    ('fare_encoder', fare_pipeline, ['Fare']),  # Preprocess 'Fare'
    ('family_size', family_size_pipeline, ['SibSp', 'Parch']),  # Preprocess 'FamilySize'
    ('embarked_encoder', embarked_pipeline, ['Embarked']),  # Preprocess 'Embarked'
    ('sex_encoder', OneHotEncoder(), ['Sex']),  # One-hot encode 'Sex'
    ('pclass_scaler', MinMaxScaler(), ['Pclass']),  # Scale 'Pclass'
], remainder='passthrough')

# Create a complete pipeline that includes preprocessing and the Random Forest classifier
rf_pipeline = Pipeline(steps=[
    ('rf_preprocessor', rf_preprocessor),  # Data preprocessing steps
    ('rf_classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Random Forest Classifier
])

# Separate features and target variable
X = data.drop('Survived', axis=1)
y = data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
rf_pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_pipeline.predict(X_test)

# Evaluate the model performance
rf_accuracy = accuracy_score(y_test, y_pred)
print(f"\nRandom Forest Model Accuracy: {rf_accuracy:.2f}")

# Confusion matrix for evaluating the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Random Forest Model Accuracy: 0.82
Confusion Matrix:
[[91 14]
 [19 55]]


# **Experiment Tracking**

In [8]:
import mlflow
import mlflow.sklearn

# Set the tracking URI and experiment name for Random Forest
mlflow.set_tracking_uri(uri="http://54.172.101.254:5000")
mlflow.set_experiment("Random Forest Experiment")

# Start a new MLflow run
with mlflow.start_run():

    # Log the hyperparameters
    mlflow.log_param("model","Random Forest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)

    # Log the accuracy metric
    mlflow.log_metric("accuracy", rf_accuracy)

    # Log the Random Forest model (use the rf_pipeline variable)
    mlflow.sklearn.log_model(rf_pipeline, "Random Forest Algorithm")


2025/11/19 13:37:35 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest Experiment' does not exist. Creating a new experiment.
2025/11/19 13:37:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 13:37:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run flawless-panda-180 at: http://54.172.101.254:5000/#/experiments/245199419062274228/runs/6c8615e2eea94d479be554955eb63df2
🧪 View experiment at: http://54.172.101.254:5000/#/experiments/245199419062274228


# Lab Task

1. Extend the experiment tracking code to log precision, recall and F1 score metrics in the Mlflow tool.
2. Register the best performing model in the Mlflow tool.
3. Perform Inference over test set using the model registered in the Step 2.
4. Perform the steps mentioned above on the following dataset.

https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease